In [ ]:
!git clone https://github.com/nasaharvest/crop-mask.git

#### Create a Config file with these steps

1. Create a file `config.yaml`
2. Follow this template to provide RoI names, and necessary file paths in your config file.

```
rois: 
  - roi_name: AlGedaref
    roi_boundary_path: '../data/roi/alqadarif_boundary.shp'
    samples_and_cropmap:
      - year: 2022
        set1_labels_path: '../data/ceo/rechecked_processed_Feb-2022---Feb-2023_ceo_samples.csv'
        set2_labels_path: '../data/ceo/rechecked_processed_Feb-2022---Feb-2023_ceo_samples.csv'
        crop_mask_path: '../data/rasters/AlQadarif_AlJazirah_2022_cropmask_epsg32636_v5_pp.tif'
      - year: 2023
        set1_labels_path: '../data/ceo/rechecked_processed_Feb-2023---Feb-2024_ceo_samples.csv'
        set2_labels_path: '../data/ceo/rechecked_processed_Feb-2023---Feb-2024_ceo_samples.csv'
        crop_mask_path: '../data/rasters/AlQadarif_AlJazirah_2023_cropmask_epsg32636_v5_pp.tif'

  - roi_name: AlJazirah
    roi_boundary_path: '../data/roi/aljazirah_boundary.shp'
    samples_and_cropmap:
      - year: 2022
        set1_labels_path: '../data/ceo/rechecked_processed_Feb-2022---Feb-2023_ceo_samples.csv'
        set2_labels_path: '../data/ceo/rechecked_processed_Feb-2022---Feb-2023_ceo_samples.csv'
        crop_mask_path: '../data/rasters/AlQadarif_AlJazirah_2022_cropmask_epsg32636_v5_pp.tif'
      - year: 2023
        set1_labels_path: '../data/ceo/rechecked_processed_Feb-2023---Feb-2024_ceo_samples.csv'
        set2_labels_path: '../data/ceo/rechecked_processed_Feb-2023---Feb-2024_ceo_samples.csv'
        crop_mask_path: '../data/rasters/AlQadarif_AlJazirah_2023_cropmask_epsg32636_v5_pp.tif'
  
output_path: '../results.csv'    
```

In [ ]:
# Install required packages
# Skip this step if you have already installed the packages in your local environment
!pip install geopandas -q
!pip install seaborn -q
!pip install rasterio -q
!pip install cartopy -q

In [ ]:
import os
import sys
import yaml

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
%cd crop-mask

In [ ]:
from src.area_utils import (
    run_area_estimates_config
)

In [ ]:
config_paths = "../config.yaml"
with open(config_paths, 'r') as file:
      config = yaml.safe_load(file)

In [ ]:
anual_estimates = run_area_estimates_config(config, show_output_charts=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_crop_area_with_confidence(anual_estimates):
    """
    This function takes in a dictionary with crop area data and plots a bar chart 
    with 95% confidence intervals for each region and year.

    Parameters:
    data (dict): A dictionary where keys are region names and values contain 
                 crop area data, confidence intervals, and years.
    """
    regions = list(anual_estimates.keys())
    years = [2022, 2023]

    crop_areas = np.array([anual_estimates[region]['crop area'] for region in regions])
    crop_ci = np.array([anual_estimates[region]['95%CI crop'] for region in regions])

    fig, ax = plt.subplots(figsize=(10, 6))

    bar_width = 0.35
    index = np.arange(len(regions))

    for i, year in enumerate(years):
        ax.bar(index + i * bar_width, crop_areas[:, i], bar_width,
               yerr=crop_ci[:, i], label=f'Year {year}', capsize=5)

    ax.set_xlabel('Region')
    ax.set_ylabel('Crop Area (hectares)')
    ax.set_title('Crop Area by Region with 95% Confidence Intervals (2022 vs 2023)')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(regions)
    ax.legend()

    ax.set_axisbelow(True)
    ax.grid(True, which='both', axis='y', linestyle='--', linewidth=0.7)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_crop_area_with_confidence(anual_estimates)